In [20]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np
import ta

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def trade_option(symbolID, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    #start_time = start_time.replace(hour=9, minute=15, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)

    ourl = 'https://groww.in/v1/api/stocks_fo_data/v4/charting_service/chart/exchange/NSE/segment/FNO/'+symbolID
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+'NIFTY'
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(ourl, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    ohlc['EMA_6'] = ohlc['close'].ewm(span=50, adjust=False).mean()
    return ohlc

In [43]:
symbol1 = 'BANKNIFTY2461950000CE'
symbol2 = 'NIFTY2462023400CE'
days_before =2
timeframe = 1
ohlc = trade_option(symbol1, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
take_profit = 10
daily_pl =0
daily_pl_limit = 10
canEnter = True
signalCount = 0

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if ohlc.iloc[i-1]['close'] - ohlc.iloc[i-1]['EMA_6'] >= 1 and canEnter == True:
        signalCount += 1
        if not position and signalCount > 1:
            position = True
            entry_price = row['open'] #change to close at run time
            trades_taken += 1
            canEnter = False
            stop_loss = 5
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['EMA_6'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

    if position:
        if row['open'] > entry_price:
            stop_loss = row['open']

    if ohlc.iloc[i-1]['close'] - ohlc.iloc[i-1]['EMA_6'] <= 1:
        canEnter = True

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*15)-(trades_taken*60)}")

Long: Entry Time: 2024-06-13 09:21:00, Entry Price: 435.45
Exit Long: Time: 2024-06-13 09:25:00, Exit Price: 425.0, P/L: -10.449999999999989
Long: Entry Time: 2024-06-13 09:26:00, Entry Price: 442.65
Exit Long: Time: 2024-06-13 09:29:00, Exit Price: 419.5, P/L: -23.149999999999977
Long: Entry Time: 2024-06-13 09:33:00, Entry Price: 430.8
Exit Long: Time: 2024-06-13 09:34:00, Exit Price: 417.2, P/L: -13.600000000000023
Long: Entry Time: 2024-06-13 10:02:00, Entry Price: 399.85
Exit Long: Time: 2024-06-13 10:03:00, Exit Price: 383.0, P/L: -16.850000000000023
Long: Entry Time: 2024-06-13 11:07:00, Entry Price: 346.2
Exit Long: Time: 2024-06-13 11:08:00, Exit Price: 364.65, P/L: 18.44999999999999
Long: Entry Time: 2024-06-13 11:31:00, Entry Price: 337.65
Exit Long: Time: 2024-06-13 11:32:00, Exit Price: 327.4, P/L: -10.25
Long: Entry Time: 2024-06-13 11:36:00, Entry Price: 333.9
Exit Long: Time: 2024-06-13 11:38:00, Exit Price: 344.9, P/L: 11.0
Long: Entry Time: 2024-06-13 11:43:00, Entry 